In [1]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [2]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components, k):
    
    label_ = label
    size = len(data)
    mod = size % 10
    
    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)

    data = np.vsplit(data, 10)
    label = u.chunkify(label, 10)
    label_ = label_[0:(size-mod)]
    pool_size = int( len(data[0])/100 * percent )
    score = 0.0
    f1 = 0.0
    mcc = 0.0
    
    if pool_size >= k:
        pred, updt, gmm = hs.handshake2(data, label_, data[0], label[0], data[1], label[1], n_components, n_features, epsilon, percent, k)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()
        predicted = pred
        acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

        # print(acc_percent, f1_percent, mcc_percent)
        # sys.exit(0)

        for i in range(1, len(data)-1):
            # print(i)
            pred, upd, gmm = hs.handshake2(data, label_, data[i], label[i], data[i+1], label[i+1], n_components, n_features, epsilon, percent, k)
            pred = np.array(pred, dtype=int)
            # print(label[i])
            pred = pred.flatten()

            # print(pred)
            acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

            predicted = np.vstack([predicted, pred])
            # print(predicted)
            # sys.exit(0)

            updt += upd
            acc_percent = np.vstack([acc_percent, acc])
            f1_percent = np.vstack([f1_percent, f1])
            mcc_percent = np.vstack([mcc_percent, mcc])


        # print(acc_percent)

        predicted = predicted.flatten()
        lab = label_[tam:]

#         tam = len(label_[0])
#         lab = label_[tam:]
#         lab = label_[len(label[0]):]


        score = accuracy_score(lab, predicted)
        f1 = f1_score(lab, predicted, average = 'macro')
        mcc = matthews_corrcoef(lab, predicted)
        # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
        # print(a,score, f1, mcc)
    return score, f1, mcc

In [3]:
poolsize = 300
clusters = 2
n_components = 2
# episilon = 0.15
step = 10

In [4]:
base = '/home/god/Documentos/tcc/datasets/'
list = ['1CDT.txt', '1CHT.txt']# '1CSurr.txt', '2CDT.txt', '2CHT.txt']# 'NOAA.txt', 'elec.txt', 'keystroke.txt']
# list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
# array_p = [30]
array_k = [50, 25, 12, 7, 5, 3, 1]

result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [5]:
dic = {}
for key, value in database.items():
    dic[int(key)] = {}
    for ep in range(0, len(array_ep)):
        for p in range(0, len(array_p)):
            for k in range(0, len(array_k)):

                adr = value
                dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

                score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_ep[ep], n_components, array_k[k])

                aux = np.hstack([score, f1, mcc])
                result = np.vstack([result, aux])

                a = str(array_ep[ep])
                b = str(array_p[p])
                c = str(array_k[k])
                name = a +' / '+ b + ' / ' + c
                dic[int(key)][name] = aux

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [6]:
for p_id, p_info in dic.items():
    print('dataset :', p_id)
    for key in p_info:
        print(key, ':', p_info[key])

dataset : 0
0.05 / 10 / 50 : [0. 0. 0.]
0.05 / 10 / 25 : [0. 0. 0.]
0.05 / 10 / 12 : [0. 0. 0.]
0.05 / 10 / 7 : [0.5625     0.55974074 0.12517048]
0.05 / 10 / 5 : [ 0.32222222  0.31944434 -0.3577631 ]
0.05 / 10 / 3 : [ 0.44444444  0.44416996 -0.1108296 ]
0.05 / 10 / 1 : [ 0.48888889  0.48822152 -0.02162194]
0.05 / 20 / 50 : [0. 0. 0.]
0.05 / 20 / 25 : [0. 0. 0.]
0.05 / 20 / 12 : [ 0.28888889  0.28       -0.43195269]
0.05 / 20 / 7 : [ 0.39722222  0.3955385  -0.20595362]
0.05 / 20 / 5 : [0.5375     0.53749911 0.07505006]
0.05 / 20 / 3 : [ 0.44166667  0.4411449  -0.11637294]
0.05 / 20 / 1 : [ 0.46527778  0.46525199 -0.06949423]
0.05 / 30 / 50 : [0. 0. 0.]
0.05 / 30 / 25 : [0. 0. 0.]
0.05 / 30 / 12 : [ 0.46944444  0.46707589 -0.06054325]
0.05 / 30 / 7 : [0.68611111 0.68548986 0.3727753 ]
0.05 / 30 / 5 : [0.67777778 0.67757626 0.35552942]
0.05 / 30 / 3 : [0.56944444 0.56888226 0.13864453]
0.05 / 30 / 1 : [0.52222222 0.52169077 0.04403484]
0.1 / 10 / 50 : [0. 0. 0.]
0.1 / 10 / 25 : [0. 0. 0.